In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary
import time
import pandas as pd

In [2]:
transform = torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 32

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:
class MyConvNet(nn.Module):
    
    def __init__(self, conv2_filters=64, fc1_features=64):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, stride=1, kernel_size=(3,3))
        self.pool1 = nn.MaxPool2d(kernel_size=(2,2), stride=2)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=conv2_filters, stride=1, kernel_size=(3,3))
        self.pool2 = nn.MaxPool2d(kernel_size=(2,2))
        self.conv3 = nn.Conv2d(in_channels=conv2_filters, out_channels=64, stride=1, kernel_size=(3,3))
        self.fc1 = nn.Linear(in_features=1024, out_features=fc1_features)
        self.fc2 = nn.Linear(in_features=fc1_features , out_features=10)
        
    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [9]:
class ModelTrainer:
    
    def __init__(self, trainloader, testloader, conv2_filters=64, fc1_features=64):
        self.trainloader = trainloader
        self.testloader = testloader
        
        self.model = MyConvNet(conv2_filters, fc1_features)
#         self.model = torch.compile(self.model)
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.001)
        
        self.mps_device = torch.device('mps')
        self.model.to(self.mps_device)
   
    def train(self):
        for epoch in range(5):

            running_loss = 0.0
            for i, (inputs, labels) in enumerate(trainloader, 0):
                inputs = inputs.to(self.mps_device, non_blocking=True)
                labels = labels.to(self.mps_device, non_blocking=True)
                
                self.optimizer.zero_grad()

                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()

                running_loss += loss.item()
            
            print(f'epoch {epoch + 1}: loss = {running_loss / 2000:.3f}')
            running_loss = 0.0
                
    def evaluate(self):
        correct, total = 0, 0
        
        with torch.no_grad():
            for images, labels in testloader:
                images = images.to(self.mps_device, non_blocking=True)
                labels = labels.to(self.mps_device, non_blocking=True)
                
                outputs = self.model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                
        return round(correct/total, 2)

In [10]:
test_values = [16, 32, 64]
accuracy_rows = {v: [] for v in test_values}
time_rows = {v: [] for v in test_values}

for conv2_filter in test_values:
    for fc1_features in test_values:
        
        trainer = ModelTrainer(trainloader, testloader, conv2_filter, fc1_features)
        start = time.time()
        trainer.train()
        end = time.time()
        
        accuracy = trainer.evaluate()
        training_time = round(end-start, 2)
        accuracy_rows[conv2_filter].append(accuracy)
        time_rows[conv2_filter].append(training_time)
        print('accuracy: ', accuracy)
        print('training time: ', training_time)

epoch 1: loss = 1.308
epoch 2: loss = 1.039
epoch 3: loss = 0.935
epoch 4: loss = 0.866
epoch 5: loss = 0.813
accuracy:  0.62
training time:  119.0
epoch 1: loss = 1.203
epoch 2: loss = 0.959
epoch 3: loss = 0.847
epoch 4: loss = 0.767
epoch 5: loss = 0.714
accuracy:  0.67
training time:  119.05
epoch 1: loss = 1.204
epoch 2: loss = 0.946
epoch 3: loss = 0.833
epoch 4: loss = 0.757
epoch 5: loss = 0.699
accuracy:  0.65
training time:  118.58
epoch 1: loss = 1.282
epoch 2: loss = 1.028
epoch 3: loss = 0.921
epoch 4: loss = 0.851
epoch 5: loss = 0.804
accuracy:  0.6
training time:  118.72
epoch 1: loss = 1.236
epoch 2: loss = 0.976
epoch 3: loss = 0.841
epoch 4: loss = 0.749
epoch 5: loss = 0.690
accuracy:  0.68
training time:  119.18
epoch 1: loss = 1.181
epoch 2: loss = 0.916
epoch 3: loss = 0.787
epoch 4: loss = 0.708
epoch 5: loss = 0.647
accuracy:  0.69
training time:  120.4
epoch 1: loss = 1.228
epoch 2: loss = 0.935
epoch 3: loss = 0.815
epoch 4: loss = 0.739
epoch 5: loss = 0.683

In [11]:
addDF = pd.DataFrame.from_dict(accuracy_rows, orient='index')
timeDF = pd.DataFrame.from_dict(time_rows, orient='index')
addDF.columns, timeDF.columns = test_values, test_values
print('accuracy:\n', addDF)
print('\training time:\n', timeDF)

accuracy:
       16    32    64
16  0.62  0.67  0.65
32  0.60  0.68  0.69
64  0.68  0.70  0.71
	raining time:
         16      32      64
16  119.00  119.05  118.58
32  118.72  119.18  120.40
64  119.73  121.01  119.03
